In [37]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [119]:
train = pd.read_csv("train_mod.csv")
test = pd.read_csv("test_mod.csv")
test.head()

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA53,PCA54,PCA55,PCA56,PCA57,PCA58,PCA59,PCA60,Target_Class,ID
0,4.353927,-1.034638,-0.816897,-3.436900,-0.254657,0.991787,-4.217148,-1.665381,4.185842,4.322290,...,1.044094,-0.666895,0.440203,0.096186,-0.323165,0.126252,0.209054,-0.112695,NaN,64252
1,1.388065,-0.254910,0.136127,-2.014415,1.883433,0.393925,1.127777,0.072659,-0.696749,-1.783014,...,1.126119,-0.912596,0.129765,0.054921,-0.322474,0.013584,0.159695,-0.394665,NaN,31278
2,-1.667943,-0.330463,-0.081393,-0.754132,0.574187,0.139522,-0.825867,-0.076025,0.057161,-1.246812,...,-0.458612,0.118556,0.224513,0.047344,-0.072047,-0.195597,0.148206,0.146459,NaN,148245
3,-1.062032,0.104152,-0.140883,-0.688352,0.674924,0.127209,-0.961733,-0.004768,-0.013219,-1.111586,...,-1.397743,0.248734,0.154301,-0.735182,-0.213937,1.615856,0.896849,-0.732519,NaN,133657
4,-1.417823,-0.067132,-0.147187,-1.062468,0.710156,0.208190,-1.143241,-0.073393,-0.056534,-1.589687,...,-0.142734,0.030734,0.122739,0.049552,-0.051880,-0.119586,-0.017300,0.022856,NaN,68463


In [120]:
test_act=pd.read_csv("Result Evaluation.csv")
test_act.head()
test_act['Target_Class']=np.where(test_act['TARGET']>0,1,0)
test_act.drop(columns='TARGET',inplace=True)
test_act.head()

,ID,Target_Class
0,64252,0
1,31278,0
2,148245,0
3,133657,0
4,68463,0


In [124]:
#Define target and ID columns:
target = 'Target_Class'
IDcol = ['ID']
from sklearn import model_selection, metrics
from sklearn.metrics import classification_report

def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
#    kfold = model_selection.KFold(n_splits=10, random_state=7)

    #Perform cross-validation:
 #   cv_train_score = model_selection.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=kfold, scoring='f1')
    print("F1 Score:",metrics.f1_score(dtrain[target].values, dtrain_predictions))
    #cv_train_score = np.sqrt(np.abs(cv_train_score))
    
    #Print model report:
    print("\nModel Report")
    print(classification_report(dtrain[target].values, dtrain_predictions))
    #print("Train CV score : ", cv_train_score)
    #print("Train RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
    
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])   
    print(classification_report(test_act['Target_Class'].values, dtest[target]))
    print("F1 Score:",metrics.f1_score(test_act['Target_Class'].values, dtest[target]))
    #cv_train_score = model_selection.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=kfold, scoring='accuracy')
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)

In [121]:
from sklearn.linear_model import LogisticRegression
predictors = [x for x in train.columns if x not in [target]+IDcol]
#print("predictors : ",predictors)
# print predictors
alg1 = LogisticRegression(verbose=1)
modelfit(alg1, train, test, predictors, target, IDcol, 'LogregPCA60.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#print(coef1)
#coef1.plot(kind='bar', title='Model Coefficients')

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]F1 Score: 0.06125356125356125

Model Report
             precision    recall  f1-score   support

        0.0       0.91      0.99      0.95     12715
        1.0       0.36      0.03      0.06      1284

avg / total       0.86      0.91      0.87     13999

             precision    recall  f1-score   support

          0       0.91      0.99      0.95      5282
          1       0.33      0.03      0.06       524

avg / total       0.86      0.91      0.87      5806

F1 Score: 0.055944055944055944


In [122]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000,max_depth=30,verbose=1,min_samples_leaf=1,min_samples_split=5, random_state=1,n_jobs=-1)
modelfit(rf, train, test, predictors, target, IDcol, 'RfPCA60.csv')

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   18.5s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0.9s finished
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   16.2s finished
[Parallel(n_jobs=8)]: Done  34 tasks      

F1 Score: 0.9374243035930561

Model Report
             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99     12715
        1.0       0.97      0.90      0.94      1284

avg / total       0.99      0.99      0.99     13999



[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:    0.5s finished


             precision    recall  f1-score   support

          0       0.92      0.97      0.95      5282
          1       0.35      0.14      0.20       524

avg / total       0.87      0.90      0.88      5806

F1 Score: 0.19891008174386923


In [133]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
dtree = DecisionTreeClassifier(criterion='gini',max_depth=10)
adb = AdaBoostClassifier(base_estimator= dtree,
        n_estimators=100,learning_rate=0.05)
print('base_estimator= dtree,n_estimators=100,learning_rate=0.05')
modelfit(adb, train, test, predictors, target, IDcol, 'ADB.csv')

base_estimator= dtree,n_estimators=100,learning_rate=0.05
F1 Score: 0.9672325305961311

Model Report
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     12715
        1.0       0.98      0.95      0.97      1284

avg / total       0.99      0.99      0.99     13999

             precision    recall  f1-score   support

          0       0.92      0.97      0.94      5282
          1       0.28      0.14      0.18       524

avg / total       0.86      0.89      0.87      5806

F1 Score: 0.1848523748395379


In [142]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(loss='deviance',learning_rate=0.05,n_estimators=100,
                                     min_samples_split=5,min_samples_leaf=1,max_depth=30 )
modelfit(gb, train, test, predictors, target, IDcol, 'gb.csv')


F1 Score: 0.9671287128712872

Model Report
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     12715
        1.0       0.98      0.95      0.97      1284

avg / total       0.99      0.99      0.99     13999

             precision    recall  f1-score   support

          0       0.92      0.95      0.94      5282
          1       0.25      0.15      0.19       524

avg / total       0.86      0.88      0.87      5806

F1 Score: 0.18867924528301885


In [135]:
import xgboost as xgb
xgb_fit = xgb.XGBClassifier(max_depth=35, n_estimators=500, learning_rate=0.05)
modelfit(xgb_fit, train, test, predictors, target, IDcol, 'xgb.csv')

F1 Score: 0.9672066376926116

Model Report
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     12715
        1.0       0.98      0.95      0.97      1284

avg / total       0.99      0.99      0.99     13999

             precision    recall  f1-score   support

          0       0.92      0.97      0.94      5282
          1       0.29      0.14      0.19       524

avg / total       0.86      0.89      0.87      5806

F1 Score: 0.18814432989690721


In [143]:
rfd=pd.read_csv("RfPCA60.csv")
regm=pd.merge(test,rfd,how='left')
testr=regm[regm.Target_Class==1]
testr.to_csv("RFtestr_mod.csv",index=False)

In [144]:
adb=pd.read_csv("ADB.csv")
adbm=pd.merge(test,adb,how='left')
testr=adbm[adbm.Target_Class==1]
testr.to_csv("ADBtestr_mod.csv",index=False)

In [145]:
xgb=pd.read_csv("xgb.csv")
xgbm=pd.merge(test,adb,how='left')
testr=xgbm[xgbm.Target_Class==1]
testr.to_csv("xgbtestr_mod.csv",index=False)

In [118]:
regm.head()

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA33,PCA34,PCA35,PCA36,PCA37,PCA38,PCA39,PCA40,Target_Class,ID
0,4.353924,-1.034666,-0.816908,-3.436869,-0.254578,0.991627,-4.217049,-1.665284,4.185490,4.322217,...,-0.660171,0.365833,-0.371812,1.026771,-1.425411,1.037520,0.494524,0.095448,0.0,64252
1,1.388060,-0.254952,0.136142,-2.014328,1.883551,0.393586,1.127976,0.073004,-0.697430,-1.783445,...,-0.811125,0.013108,-0.146968,0.689167,-0.538192,0.021378,0.302274,0.250053,0.0,31278
2,-1.667940,-0.330461,-0.081428,-0.754170,0.574090,0.139786,-0.826036,-0.076139,0.057482,-1.246424,...,0.152686,0.005735,0.062138,0.157630,0.267558,-0.314182,-0.500254,0.159355,0.0,148245
3,-1.062029,0.104153,-0.140874,-0.688492,0.674738,0.127362,-0.962028,-0.005004,-0.013685,-1.111548,...,0.539738,-0.558247,0.328491,0.423368,0.647217,-0.008885,-0.429041,0.009484,0.0,133657
4,-1.417823,-0.067132,-0.147186,-1.062483,0.710155,0.208195,-1.143242,-0.073451,-0.056441,-1.589555,...,-0.016985,-0.021716,0.013131,0.045964,0.182837,-0.040227,-0.067553,0.086005,0.0,68463


In [112]:
testr.head()

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA33,PCA34,PCA35,PCA36,PCA37,PCA38,PCA39,PCA40,Target_Class,ID


In [109]:
adb=pd.read_csv("ADB.csv")
adbm=pd.merge(test,adb,how='left')

In [110]:
adbm[adbm.Target_Class==1]

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA33,PCA34,PCA35,PCA36,PCA37,PCA38,PCA39,PCA40,Target_Class,ID


In [73]:
testr=adbm[adbm.Target_Class==1]
testr.to_csv("testr_mod.csv",index=False)

In [136]:
DL=pd.read_csv("DL.csv")

In [141]:
DL[DL.Target_Class>0.3].shape

(136, 2)